In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import gc
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix




In [2]:
# get working directory
wd = os.path.dirname(os.getcwd())
# wd = "N:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance"
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  n:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance


In [3]:
from utils.loader_steve import load_data, load_labels , load_all_X_y

df, labels = load_all_X_y( data= "flattened", )
keys = list(df.keys())

dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

lab_1 = pd.read_csv('data/processed/labels_q1-3.csv', dtype=dtypes_labels)
lab_2 = pd.read_csv('data/processed/labels_q4-13.csv', dtype=dtypes_labels)
lab_3= pd.read_csv('data/processed/labels_q14-18.csv', dtype=dtypes_labels)

labels_dict = { "0_4": lab_1,
        "5_12": lab_2,
        "13_22": lab_3
}

In [4]:
keys = list(df.keys())
labels_dict = {
    "0_4": lab_1,
    "5_12": lab_2,
    "13_22": lab_3
}

# Create an empty dictionary to store the output DataFrames
output_dict = {}

# Iterate over each key in the original dictionary
for key in keys:
    train_data = df[key]
    train_data = train_data.sort_values('session_id_1')
    labels = labels_dict[key]
    labels.rename(columns={'session': 'session_id_1'}, inplace=True)
    labels = labels.sort_values('session_id_1')
    merged_df = pd.merge(train_data, labels, on="session_id_1", how="inner")
    merged_df = merged_df.drop(columns=[col for col in merged_df.columns if 'session' in col])
    grouped_df = merged_df.sort_values('q')
    
    # Group the merged DataFrame by the "q" column
    grouped_by_level = grouped_df.groupby('q')
    
    # Iterate over each unique level in the grouped DataFrame
    for level, group_data in grouped_by_level:
        # Create a unique key for the output dictionary
        output_key = f"{level}"
        # Drop the "level_group" and "q" columns from the group_data DataFrame
        group_data = group_data.drop(columns=["level_group_1", "q"])
        # Store the group_data DataFrame in the output dictionary
        output_dict[output_key] = group_data



In [5]:
from sklearn.model_selection import train_test_split

# Split ratio for train, test, and validation sets
train_ratio = 0.7
test_ratio = 0.2
val_ratio = 0.1

# Create dictionaries for train, test, and validation sets
train_dict = {}
test_dict = {}
val_dict = {}

# Iterate over each key in the output_dict
for key, data in output_dict.items():
    # Split data into train and remaining data (test + validation)
    data_train, data_remaining = train_test_split(data, test_size=(test_ratio + val_ratio), random_state=42)
    
    # Split remaining data into test and validation
    data_test, data_val = train_test_split(data_remaining, test_size=val_ratio / (test_ratio + val_ratio), random_state=42)
    
    # Store the data in respective dictionaries
    train_dict[key] = data_train
    test_dict[key] = data_test
    val_dict[key] = data_val


In [ ]:
'''check = train_dict["1"]
vars_to_keep = ['train_dict', 'test_dict', 'val_dict']
all_vars = list(globals().keys())
vars_to_drop = [var for var in all_vars if var not in vars_to_keep]

for var in vars_to_drop:
    del globals()[var]'''

In [ ]:
'''import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm  # for progress tracking

# Dictionary to store results
results_dict = {}
test_data_dict = {}
#index to stop early
index = 0
limit = 1
# Iterate over each key (model) in the output_dict
for key, data in tqdm(output_dict.items(), desc='GridSearch Progress', total=len(output_dict)):
    X = data.drop("correct", axis=1)  # Features
    y = data["correct"]  # Labels

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    test_data_dict[key] = {
        "X_test": X_test,
        "y_test": y_test
    }

    # Define hyperparameter values to search for each model
    param_grid = {
        'max_depth': [3, 6, 9],
        'learning_rate': [0.1, 0.01, 0.001],
        'n_estimators': [100, 200, 300],
        'gamma': [0, 1, 5],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'scale_pos_weight': [1, 5, 10]
    }

    # Create XGBoost classifier
    model = xgb.XGBClassifier()

    # Perform grid search with cross-validation for the current model
    grid_search = GridSearchCV(model, param_grid, cv=10, scoring='f1')
    grid_search.fit(X_train, y_train)

    # Get the grid search results for each hyperparameter combination
    cv_results = grid_search.cv_results_

    # Store results for the current model
    results_dict[key] = {
        "cv_results": cv_results,
        "best_params": grid_search.best_params_,
        "best_score": grid_search.best_score_,
        "f1_score": f1_score(y_test, grid_search.predict(X_test))
    }
    index += 1
    if index == limit:
        break
'''

In [ ]:
import xgboost as xgb
import numpy as np
from hyperopt import fmin, hp, tpe, Trials
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd

# Define search space for hyperparameters
param_space = {
    'max_depth': hp.choice('max_depth', [3, 6, 9]),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300]),
    'gamma': hp.choice('gamma', [0, 1, 5]),
    'subsample': hp.uniform('subsample', 0.8, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.8, 1.0),
    'scale_pos_weight': hp.choice('scale_pos_weight', [1, 5, 10])
}

# Dictionary to store results
results_dict = {}
test_data_dict = {}
best_models = {}  # Dictionary to store best models

# Define the objective function for hyperopt
def objective(params):
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = f1_score(y_val, y_pred)
    return -score  # maximize F1 score

# Iterate over each key (model) in the train_dict
for key, train_data in tqdm(train_dict.items()):
    val_data = val_dict[key]
    test_data = test_dict[key]

    X_train = train_data.drop("correct", axis=1)  # Features for training
    y_train = train_data["correct"]  # Labels for training

    X_val = val_data.drop("correct", axis=1)  # Features for validation
    y_val = val_data["correct"]  # Labels for validation
    
    X_test = test_data.drop("correct", axis=1)  # Features for testing
    y_test = test_data["correct"]  # Labels for testing

    # Run hyperparameter optimization using hyperopt
    trials = Trials()
    best_params = fmin(objective, param_space, algo=tpe.suggest, max_evals=100, trials=trials)

    # Fit the model with the best hyperparameters using train and validation data
    model = xgb.XGBClassifier(**best_params)
    X_train_val = pd.concat([X_train, X_val])
    y_train_val = pd.concat([y_train, y_val])
    model.fit(X_train_val, y_train_val)

    # Evaluate on the test set
    y_pred = model.predict(X_test)
    best_score = f1_score(y_test, y_pred)

    # Store results for the current model
    results_dict[key] = {
        "best_params": best_params,
        "best_score": best_score
    }
    
    # Store the best model
    best_models[key] = model


In [ ]:
# Iterate over each key-value pair in results_dict
for key, value in results_dict.items():
    best_score = value["best_score"]
    print(f"Model: {key}, Best Score: {best_score}")

In [10]:
#xgboost
# import xgboost as xgb
import pandas as pd
from sklearn.metrics import f1_score

# Select the data for model training
train_data = train_dict['10']
X_train = train_data.drop("correct", axis=1)
y_train = train_data["correct"]

# Select the data for additional training (using validation set)
val_data = val_dict['10']
X_train_val = pd.concat([X_train, val_data.drop("correct", axis=1)])
y_train_val = pd.concat([y_train, val_data["correct"]])

# Select the data for testing
test_data = test_dict['10']
X_test = test_data.drop("correct", axis=1)
y_test = test_data["correct"]

# Define hyperparameters
params = {
    'max_depth': 9,
    'learning_rate': 0.01,
    'n_estimators': 200,
    'gamma': 1,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'scale_pos_weight': 5
}

# Train the model with the combined training and validation data
model = xgb.XGBClassifier(**params)
model.fit(X_train_val, y_train_val)

# Evaluate on the test set
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred)

print("F1 Score:", f1)


F1 Score: 0.6818452380952381


In [11]:
import xgboost as xgb
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score

# Select the data for model training
train_data = train_dict['10']
X_train = train_data.drop("correct", axis=1)
y_train = train_data["correct"]

# Select the data for additional training (using validation set)
val_data = val_dict['10']
X_val = val_data.drop("correct", axis=1)
y_val = val_data["correct"]

# Select the data for testing
test_data = test_dict['10']
X_test = test_data.drop("correct", axis=1)
y_test = test_data["correct"]

# Perform SVD-based dimensionality reduction
svd = TruncatedSVD(n_components=50, random_state=42)
X_train_svd = svd.fit_transform(X_train)
X_val_svd = svd.transform(X_val)
X_test_svd = svd.transform(X_test)

# Define hyperparameters
params = {
    'max_depth': 9,
    'learning_rate': 0.01,
    'n_estimators': 200,
    'gamma': 1,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'scale_pos_weight': 5
}

# Train the model with the combined training and validation data after dimensionality reduction
model = xgb.XGBClassifier(**params)
X_train_val_svd = pd.concat([pd.DataFrame(X_train_svd), pd.DataFrame(X_val_svd)])
y_train_val = pd.concat([y_train, y_val])
model.fit(X_train_val_svd, y_train_val)

# Evaluate on the test set after dimensionality reduction
y_pred = model.predict(X_test_svd)
f1 = f1_score(y_test, y_pred)

print("F1 Score:", f1)


F1 Score: 0.6765710889912345


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy

# Function to create the neural network model
def create_model(optimizer='adam', learning_rate=0.001, dropout_rate=0.2):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(256, activation='relu'),
        Dropout(dropout_rate),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss=BinaryCrossentropy())
    return model

# Create a KerasClassifier wrapper for Scikit-learn compatibility
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

# Define hyperparameters to search over
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.01, 0.001],
    'dropout_rate': [0.2, 0.3, 0.4],
    'batch_size': [32, 64],
}

# Perform random search with cross-validation
random_search = RandomizedSearchCV(model, param_distributions=param_grid, scoring='f1', n_iter=10, cv=3)
random_search.fit(X_train_val, y_train_val)

# Get the best model and evaluate on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
f1 = f1_score(y_test, y_pred)

print("Best F1 Score:", random_search.best_score_)
print("F1 Score on Test Set:", f1)
print("Best Hyperparameters:", random_search.best_params_)


C:\Users\steph\AppData\Local\Temp\ipykernel_2096\563449181.py:32: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)


148/148 [==============================] - 0s 844us/step
Best F1 Score: 0.6720600783200504
F1 Score on Test Set: 0.6691145318457845
Best Hyperparameters: {'optimizer': 'adam', 'learning_rate': 0.001, 'dropout_rate': 0.4, 'batch_size': 32}


In [25]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tqdm import tqdm

# Select the data for model training
train_data = train_dict['10']
X_train = train_data.drop("correct", axis=1)
y_train = train_data["correct"]

# Select the data for additional training (using validation set)
val_data = val_dict['10']
X_val = val_data.drop("correct", axis=1)
y_val = val_data["correct"]

# Select the data for testing
test_data = test_dict['10']
X_test = test_data.drop("correct", axis=1)
y_test = test_data["correct"]

# Perform SVD-based dimensionality reduction
svd = TruncatedSVD(n_components=50, random_state=42)
X_train_val = pd.concat([X_train, X_val])
X_train_val_svd = svd.fit_transform(X_train_val)
X_test_svd = svd.transform(X_test)

# Define the bigger neural network model
def create_bigger_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_val_svd.shape[1],)),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(512, activation='relu'),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy())
    return model

# Train the bigger neural network model with the combined training and validation data after dimensionality reduction
bigger_model = create_bigger_model()
bigger_model.fit(X_train_val_svd, pd.concat([y_train, y_val]), epochs=50, batch_size=32, verbose=1)

# Evaluate on the test set after dimensionality reduction
y_pred_bigger = bigger_model.predict(X_test_svd)
y_pred_bigger = (y_pred_bigger > 0.5).astype(int)
f1_bigger = f1_score(y_test, y_pred_bigger)

print("F1 Score (Bigger Model):", f1_bigger)


Epoch 1/50
590/590 [==============================] - 4s 6ms/step - loss: 173.1051
Epoch 2/50
590/590 [==============================] - 3s 6ms/step - loss: 149.2191
Epoch 3/50
590/590 [==============================] - 3s 6ms/step - loss: 139.4110
Epoch 4/50
590/590 [==============================] - 3s 6ms/step - loss: 84.4823
Epoch 5/50
590/590 [==============================] - 3s 6ms/step - loss: 111.0638
Epoch 6/50
590/590 [==============================] - 3s 6ms/step - loss: 260.8571
Epoch 7/50
590/590 [==============================] - 3s 6ms/step - loss: 94.1614
Epoch 8/50
590/590 [==============================] - 3s 5ms/step - loss: 142.6135
Epoch 9/50
590/590 [==============================] - 3s 6ms/step - loss: 69.1689
Epoch 10/50
590/590 [==============================] - 3s 6ms/step - loss: 104.0839
Epoch 11/50
590/590 [==============================] - 3s 6ms/step - loss: 163.8286
Epoch 12/50
590/590 [==============================] - 3s 6ms/step - loss: 113.4802
Epoc